In [1]:
!pip install control
import numpy as np
import control
from misc_tools.print_latex import print_tex


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


input example : 
>>> arr_T = np.array([[r'\vec{v}_1', r'\vec{v}_2']]).T
>>> print_tex(arr_T,'=', np.arange(1,5).reshape(2,-1)/4, r'; symbols: \otimes, \cdot,\times')
output: 


<IPython.core.display.Math object>

# Controllability

## How to make system controllable?

In last chapter we determined that some system can be stable and non-stable based on matrix A eigenvalues. 
$$\dot{ \vec x }  \approx A \vec x$$
This can be interpreted as: "if we placed system in stable point (or near it?), will it stay/return to stable point or move away".

Goal of control, is to "modify" system dynamics to passively or actively guide trajectory of a state.

We can do it by adding "control" term $B \vec u$: 
$$ \dot{ \vec x }  = A \vec x + B \vec u $$ 
where $\vec u$ represents $q$ control parameters ("knobs"), which can be a scalar or a vector, depending on requirements.

$$\vec x \in \R^n \ ; \ A \in  \R^{n \times n}\ ; \ \vec u \in \R^q \ ; \ B \in \R^{n \times q}$$

$A$ represents natural system dynamics and $B$ represents properties of control (actuators). Often you dont get to choose them.



### Linear control

One example is to employ linear control:
$$\vec u = -K \vec x $$
which produces new system dynamics
$$ \dot{ \vec x } = (A - BK) \vec x$$ 
for which we can "shape" K that will make our system controllable.

## Determining if system is controllable

### Manually, by intuition

suppose we have simple system with decoupled states and add following control with 1 "knob" 
$B \vec u = \begin{bmatrix}
 0 \\  1
\end{bmatrix}
u$:
$$\begin{bmatrix}
\dot x_1 \\ \dot x_2
\end{bmatrix}=
\begin{bmatrix}
1 & 0 \\ 0 & 2
\end{bmatrix}
\begin{bmatrix}
 x_1 \\  x_2
\end{bmatrix}
+
\begin{bmatrix}
 0 \\  1
\end{bmatrix}
u
$$
Is system controllable? No. We can see that $u$ can only influence dynamics of $\dot x_2$. It is easier to see if we rewrite dynamics as system of equations
$$\begin{cases}
\dot x_1 = x_1 \\
\dot x_2 = 2x_2 + u
\end{cases}$$

On the other hand, if we choose 
$B \vec u = \begin{bmatrix}
 1 & 0 \\  0 & 1
\end{bmatrix}
\begin{bmatrix}
 u_1 \\ u_2
\end{bmatrix}
$

our "knobs" can affect dynamics of both states:
$$\begin{cases}
\dot x_1 = x_1 + u_1\\
\dot x_2 = 2x_2 + u_2
\end{cases}$$

Although there can be a case if system is "coupled", simple first control will work
$$\begin{bmatrix}
\dot x_1 \\ \dot x_2
\end{bmatrix}=
\begin{bmatrix}
1 & 1 \\ 0 & 2
\end{bmatrix}
\begin{bmatrix}
 x_1 \\  x_2
\end{bmatrix}
+
\begin{bmatrix}
 0 \\  1
\end{bmatrix}
u
$$
or when viewed as
$$\begin{cases}
\dot x_1 = x_1 + x_2\\
\dot x_2 = 2x_2 + u
\end{cases}$$
We see that although $u$ does not directly affect $\dot x_1$, it is "connected" via $x_2$


### By examining controllability matrix

Controllability matrix is defined as
$$C = \begin{bmatrix}
B & AB & A^2B & \dots & A^{n-1}B
\end{bmatrix}$$

If matrix "C" has n linearly independent columns (full column rank), then system is controllable

We can test our simple systems and construct $C$ manually

$$n = 2; q = 1$$
$$B = \begin{bmatrix}
0 \\ 1
\end{bmatrix}$$
$$AB = A^{n-1}B=
\begin{bmatrix}
1 & 0 \\ 0 & 2
\end{bmatrix}
\begin{bmatrix}
0 \\ 1
\end{bmatrix}=
\begin{bmatrix}
0 \\ 2
\end{bmatrix}$$
so
$$C_1 = 
\begin{bmatrix}
B & AB
\end{bmatrix}=
\begin{bmatrix}
0 & 0 \\ 1 & 2
\end{bmatrix}$$
has two linearly dependent columns (rank = $1$), and for coupled case
$$C_2 = 
\begin{bmatrix}
0 & 1 \\ 1 & 2
\end{bmatrix}$$
rank is $2$, and we conclude that it is truly first system that is impossible to control.

We can also do it via control library:

In [2]:
A1 = np.array([[1,0],[0,2]])
A2 = np.array([[1,1],[0,2]])
B = np.array([[0,1]]).T

C1 = control.ctrb(A1,B)
C2 = control.ctrb(A2,B)
print_tex("C_1 = ", C1, " ; rank = ", np.linalg.matrix_rank(C1))
print_tex("C_2 = ", C2, " ; rank = ", np.linalg.matrix_rank(C2))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [3]:
np.linalg.eigvals(A2)

array([1., 2.])